In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
 # to be created:
    # time based variable: hour_of_day, day_of_week, month 
    # seasonality: May to September is summer, October to April is winter
    # weather: wind_speed_per_city
    # time series data: pool_price_lagged_values_till_24 hours, AIL_lagged_values_till_24 hours